In [31]:
# Cell 1
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow', region='us-east-1', version='2.12.0', image_scope='training', instance_type='ml.c5.4xlarge')


INFO:sagemaker.image_uris:Defaulting to only available Python version: py310


'763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310'

In [32]:
# Cell 2
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com


WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [33]:
# Cell 3
!docker pull 763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310


2.12.0-cpu-py310: Pulling from tensorflow-training
Digest: sha256:e656990fae3aff490156924ed05b43ac8e01d02097de068f8df86cbbd8cbf4af
Status: Image is up to date for 763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310
763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310


In [34]:
# Cell 4
!docker images


REPOSITORY                                                         TAG                IMAGE ID       CREATED          SIZE
648633553332.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training   mondo              8e232d4a241b   12 minutes ago   6.13GB
tensorflow-training                                                mondo              8e232d4a241b   12 minutes ago   6.13GB
648633553332.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training   <none>             bb2604e3dd92   38 minutes ago   6.13GB
763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training   2.12.0-cpu-py310   7f67779c9e81   9 months ago     6.13GB


In [35]:
# Cell 5
!docker build -t tensorflow-training:mondo -f Dockerfile.train .


[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.1s (8/9)                                          docker:default
 => [internal] load build definition from Dockerfile.train                 0.0s
 => => transferring dockerfile: 372B                                       0.0s
 => [internal] load metadata for 763104351884.dkr.ecr.us-east-1.amazonaws  0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [1/4] FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-tr  0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 3.07kB                                        0.0s
 => CACHED [2/4] COPY requirements.txt /opt/ml/code/requirements.txt       0.0s
 => CACHED [3/4] RUN pip install -r /opt/ml/code/requirements.txt          0.0s
 => [4/4] ADD ./main.py /opt/ml/code/mai

In [36]:
# Cell 6
!docker images


REPOSITORY                                                         TAG                IMAGE ID       CREATED          SIZE
tensorflow-training                                                mondo              4f45dbf42661   2 seconds ago    6.13GB
648633553332.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training   mondo              8e232d4a241b   13 minutes ago   6.13GB
648633553332.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training   <none>             bb2604e3dd92   39 minutes ago   6.13GB
763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training   2.12.0-cpu-py310   7f67779c9e81   9 months ago     6.13GB


In [37]:
%%sh
# Specifica il nome dell'immagine
image_name=tensorflow-training
echo "image_name: ${image_name} ######################"

# Ottieni l'Account ID
account=$(aws sts get-caller-identity --query Account --output text)
echo "account: ${account} ######################"

# Ottieni la regione configurata (predefinita us-east-1 se non definita)
region=$(aws configure get region)
if [ -z "$region" ]; then
    region="us-east-1"
fi
echo "region: ${region} ######################"

# Crea il nome completo dell'immagine
fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:mondo"
echo "fullname: ${fullname} ######################"

# Crea il repository ECR se non esiste
aws ecr describe-repositories --repository-names "${image_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${image_name}"
    echo "Repository ${image_name} creato."
else
    echo "Repository ${image_name} già esistente."
fi

# Tag dell'immagine Docker
docker tag ${image_name}:mondo ${fullname}


image_name: tensorflow-training ######################
account: 648633553332 ######################
region: us-east-1 ######################
fullname: 648633553332.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:mondo ######################
Repository tensorflow-training già esistente.


In [19]:
%%sh
account=$(aws sts get-caller-identity --query Account --output text)
region=$(aws configure get region)
if [ -z "$region" ]; then
    region="us-east-1"
fi

aws ecr get-login-password --region ${region} | docker login --username AWS --password-stdin ${account}.dkr.ecr.${region}.amazonaws.com


WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded


In [38]:
%%sh
# Recupera le variabili necessarie
account=$(aws sts get-caller-identity --query Account --output text)
region=$(aws configure get region)
if [ -z "$region" ]; then
    region="us-east-1"
fi
image_name=tensorflow-training
fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:mondo"

# Effettua il push dell'immagine
docker push ${fullname}


The push refers to repository [648633553332.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training]
f551d80c1ee4: Preparing
d7b9f32dbe56: Preparing
063e2c83c9df: Preparing
f8d322878ecf: Preparing
f7f7db4f6696: Preparing
b8a6f529ae49: Preparing
00c775f13e59: Preparing
c36a0c6c24f9: Preparing
a8a9a1e64371: Preparing
c84629631b13: Preparing
68983f03efd0: Preparing
95ba9b75d5ee: Preparing
c3c5370de786: Preparing
7088a16f1a7a: Preparing
07adfc8740d1: Preparing
7688292691fc: Preparing
96186672bf9d: Preparing
b8b5607be3da: Preparing
3d60afc9b360: Preparing
f392f934e4df: Preparing
831980a2a6eb: Preparing
b8d9a0538fa4: Preparing
e6b16d109c54: Preparing
c2a8f0a068df: Preparing
6f5f443b7673: Preparing
75de37a36861: Preparing
6fc7ecb4bd81: Preparing
440134b60c81: Preparing
f77c07d5d0ab: Preparing
3ca1e6c8e003: Preparing
fbeeab822c1c: Preparing
eb763609315f: Preparing
28da0445c449: Preparing
c36a0c6c24f9: Waiting
a8a9a1e64371: Waiting
b8d9a0538fa4: Waiting
c84629631b13: Waiting
e6b16d109c54: Waiting
6

In [39]:
# Cell 10
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import os

# Ottieni il ruolo di esecuzione
role = get_execution_role()

# Definisci gli iperparametri
hyperparameters = {
    'epochs': 10  # Aumenta o diminuisci a seconda del modello
}

# Recupera l'Account ID e la Regione usando comandi shell
account_id = !aws sts get-caller-identity --query Account --output text
account_id = account_id[0]

region = !aws configure get region
region = region[0] if region else 'us-east-1'

image_name = 'tensorflow-training'
tag = 'mondo'
#fullname = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{image_name}:{tag}"
fullname = "648633553332.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:mondo"


# Crea l'estimatore con l'immagine Docker personalizzata
estimator = Estimator(
    image_uri=fullname,
    role=role,
    instance_count=1,
    instance_type='ml.c5.4xlarge',  # Puoi scegliere il tipo di istanza appropriato
    hyperparameters=hyperparameters,
    output_path='s3://itsar123-bianchini/data/output/'  # Sostituisci con il tuo bucket S3
)

# Avvia l'addestramento sul dataset degli esopianeti
print('##### INIZIO ADDDESTRAMENTO #####')
estimator.fit('s3://itsar123-bianchini/data/input/nasa_exoplanets.csv')  # Sostituisci con il tuo percorso S3
print('##### ADDDESTRAMENTO COMPLETATO #####')


INFO:sagemaker:Creating training-job with name: tensorflow-training-2024-11-04-16-29-11-763


##### INIZIO ADDDESTRAMENTO #####
2024-11-04 16:29:13 Starting - Starting the training job...
2024-11-04 16:29:26 Starting - Preparing the instances for training...
2024-11-04 16:30:08 Downloading - Downloading the training image......
2024-11-04 16:30:54 Training - Training image download completed. Training in progress..2024-11-04 16:31:17.194118: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX512F, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-04 16:31:19,111 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2024-11-04 16:31:19,112 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-04 16:31:19,113 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-11-04 16:31:19,123 sa